In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving MahaSent_ST_Test.csv to MahaSent_ST_Test.csv
Saving MahaSent_ST_Train.csv to MahaSent_ST_Train.csv
Saving MahaSent_ST_Val.csv to MahaSent_ST_Val.csv
Saving tweets-test_GT.csv to tweets-test_GT.csv
Saving tweets-test_PT.csv to tweets-test_PT.csv
Saving tweets-train_GT.csv to tweets-train_GT.csv
Saving tweets-train_PT.csv to tweets-train_PT.csv
Saving tweets-valid_GT.csv to tweets-valid_GT.csv
Saving tweets-valid_PT.csv to tweets-valid_PT.csv


In [ ]:
# PT
pt_train_df = pd.read_csv("tweets-train_PT.csv")
pt_val_df   = pd.read_csv("tweets-valid_PT.csv")
pt_test_df  = pd.read_csv("tweets-test_PT.csv")

# ST
st_train_df = pd.read_csv("MahaSent_ST_Train.csv")
st_val_df   = pd.read_csv("MahaSent_ST_Val.csv")
st_test_df  = pd.read_csv("MahaSent_ST_Test.csv")

# GT
gt_train_df = pd.read_csv("tweets-train_GT.csv")
gt_val_df   = pd.read_csv("tweets-valid_GT.csv")
gt_test_df  = pd.read_csv("tweets-test_GT.csv")


In [ ]:
print("PT Train:")
print(pt_train_df.head())
print("\nST Train:")
print(st_train_df.head())
print("\nGT Train:")
print(gt_train_df.head())

PT Train:
                                               tweet  label
0  ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...     -1
1  सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...     -1
2  उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...     -1
3  आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...      1
4  बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...     -1

ST Train:
   Unnamed: 0                                  marathi_text  label
0           0                              मी तो मूर्ख आहे.     -1
1           1  आईने आत्महत्या केली आणि माझे वडील पळून गेले.     -1
2           2                   - त्यांनी फक्त जॉयला मारले?     -1
3           3            मला माफ करा मी आज तिथेच केले नाही.     -1
4           4                            नाही, नक्कीच नाही.     -1

GT Train:
                                               tweet  label  political
0          होता होता राहीलेला  निवडणूक मारो मर्ज़ीभई     -1          0
1                         खरा लखोबा तर हा बोबडाच आहे     -1     

In [ ]:
# RENAME COLUMNS → unify to 'text'
pt_train_df = pt_train_df.rename(columns={"tweet": "text"})
st_train_df = st_train_df.rename(columns={"marathi_text": "text"})
gt_train_df = gt_train_df.rename(columns={"tweet": "text"})

pt_train_df = pt_train_df[["text", "label"]]
st_train_df = st_train_df[["text", "label"]]
gt_train_df = gt_train_df[["text", "label"]]

In [ ]:
# MERGE TRAIN: PT + ST + GT
train_df = pd.concat([pt_train_df, st_train_df, gt_train_df], ignore_index=True)
train_df = train_df.sample(frac=1, random_state=42)

In [ ]:
# VALIDATION MERGE
pt_val_df = pt_val_df.rename(columns={"tweet": "text"})
st_val_df = st_val_df.rename(columns={"marathi_text": "text"})
gt_val_df = gt_val_df.rename(columns={"tweet": "text"})

val_df = pd.concat([
    pt_val_df[["text", "label"]],
    st_val_df[["text", "label"]],
    gt_val_df[["text", "label"]],
], ignore_index=True)

val_df = val_df.sample(frac=1, random_state=42)

In [ ]:
# TEST MERGE
# ================================
pt_test_df = pt_test_df.rename(columns={"tweet": "text"})
st_test_df = st_test_df.rename(columns={"marathi_text": "text"})
gt_test_df = gt_test_df.rename(columns={"tweet": "text"})

test_df = pd.concat([
    pt_test_df[["text", "label"]],
    st_test_df[["text", "label"]],
    gt_test_df[["text", "label"]],
], ignore_index=True)

test_df = test_df.sample(frac=1, random_state=42)


In [ ]:
# CLEAN LABELS + TEXT
for df in [train_df, val_df, test_df]:
    df["label"] = df["label"].astype(int) + 1   # -1→0, 0→1, 1→2
    df["text"] = df["text"].astype(str).fillna("")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

In [ ]:
model_name = "l3cube-pune/marathi-bert-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(batch):
    texts = [str(x) for x in batch["text"]]
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_encoded = train_dataset.map(tokenize_fn, batched=True)
val_encoded   = val_dataset.map(tokenize_fn, batched=True)
test_encoded  = test_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/36114 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./pt_st_gt_marathi_bert_v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    fp16=True,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3780730124.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,0.477500,0.449213,0.821778,0.820340,0.821778,0.820338
2,0.387000,0.452049,0.824889,0.823557,0.824889,0.823872
3,0.290700,0.484929,0.825556,0.824618,0.825556,0.824960


TrainOutput(global_step=6774, training_loss=0.4466396562110579, metrics={'train_runtime': 1206.8006, 'train_samples_per_second': 89.776, 'train_steps_per_second': 5.613, 'total_flos': 7126558475871744.0, 'train_loss': 0.4466396562110579, 'epoch': 3.0})

In [ ]:
test_metrics = trainer.evaluate(test_encoded)
print("Combined PT + ST + GT Test metrics:", test_metrics)

Combined PT + ST + GT Test metrics: {'eval_loss': 0.49982914328575134, 'eval_accuracy': 0.8215238095238095, 'eval_precision_macro': 0.8209015914739415, 'eval_recall_macro': 0.8215238095238094, 'eval_f1_macro': 0.8211114248953679, 'eval_runtime': 16.3634, 'eval_samples_per_second': 320.837, 'eval_steps_per_second': 20.106, 'epoch': 3.0}


In [ ]:
# ----- PT -----
pt_test_df_processed = pt_test_df.copy()
pt_test_df_processed["label"] = pt_test_df_processed["label"].astype(int) + 1
pt_test_df_processed["text"] = pt_test_df_processed["text"].astype(str).fillna("")
pt_test_dataset = Dataset.from_pandas(pt_test_df_processed)
pt_test_encoded = pt_test_dataset.map(tokenize_fn, batched=True)
pt_test_metrics = trainer.evaluate(pt_test_encoded)
print("PT Test metrics:", pt_test_metrics)

Map:   0%|          | 0/2250 [00:00<?, ? examples/s]

PT Test metrics: {'eval_loss': 0.4135251045227051, 'eval_accuracy': 0.8555555555555555, 'eval_precision_macro': 0.8561879087802162, 'eval_recall_macro': 0.8555555555555556, 'eval_f1_macro': 0.8551091759885286, 'eval_runtime': 9.0623, 'eval_samples_per_second': 248.282, 'eval_steps_per_second': 15.559, 'epoch': 3.0}


In [ ]:
# ----- ST -----
st_test_df_processed = st_test_df.copy()
st_test_df_processed["label"] = st_test_df_processed["label"].astype(int) + 1
st_test_df_processed["text"] = st_test_df_processed["text"].astype(str).fillna("")
st_test_dataset = Dataset.from_pandas(st_test_df_processed)
st_test_encoded = st_test_dataset.map(tokenize_fn, batched=True)
st_test_metrics = trainer.evaluate(st_test_encoded)
print("ST Test metrics:", st_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

ST Test metrics: {'eval_loss': 0.5572214722633362, 'eval_accuracy': 0.7973333333333333, 'eval_precision_macro': 0.7970040962621607, 'eval_recall_macro': 0.7973333333333333, 'eval_f1_macro': 0.7971603598990917, 'eval_runtime': 4.7485, 'eval_samples_per_second': 315.891, 'eval_steps_per_second': 19.796, 'epoch': 3.0}


In [ ]:
gt_test_df_processed = gt_test_df.copy()
gt_test_df_processed["label"] = gt_test_df_processed["label"].astype(int) + 1
gt_test_df_processed["text"] = gt_test_df_processed["text"].astype(str).fillna("")
gt_test_dataset = Dataset.from_pandas(gt_test_df_processed)
gt_test_encoded = gt_test_dataset.map(tokenize_fn, batched=True)
gt_test_metrics = trainer.evaluate(gt_test_encoded)
print("GT Test metrics:", gt_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

GT Test metrics: {'eval_loss': 0.5718926787376404, 'eval_accuracy': 0.7946666666666666, 'eval_precision_macro': 0.7959459791196508, 'eval_recall_macro': 0.7946666666666666, 'eval_f1_macro': 0.7952261619546142, 'eval_runtime': 3.764, 'eval_samples_per_second': 398.512, 'eval_steps_per_second': 24.973, 'epoch': 3.0}


In [ ]:
uploaded = files.upload()

Saving MahaSent_MR_Test.csv to MahaSent_MR_Test.csv


In [ ]:

mr_test_df = pd.read_csv("MahaSent_MR_Test.csv")

mr_test_df_processed = mr_test_df.copy()
mr_test_df_processed = mr_test_df_processed.rename(columns={"marathi_sentence": "text"})
mr_test_df_processed["label"] = mr_test_df_processed["label"].astype(int) + 1
mr_test_df_processed["text"] = mr_test_df_processed["text"].astype(str).fillna("")

mr_test_dataset = Dataset.from_pandas(mr_test_df_processed)
mr_test_encoded = mr_test_dataset.map(tokenize_fn, batched=True)

mr_test_metrics = trainer.evaluate(mr_test_encoded)
print("MR Test metrics:", mr_test_metrics)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

MR Test metrics: {'eval_loss': 0.7525001764297485, 'eval_accuracy': 0.7346666666666667, 'eval_precision_macro': 0.734034757475866, 'eval_recall_macro': 0.7346666666666667, 'eval_f1_macro': 0.7320766131425526, 'eval_runtime': 3.6631, 'eval_samples_per_second': 409.488, 'eval_steps_per_second': 25.661, 'epoch': 3.0}
